In [1]:
#####Note:run in python2######
# same initialization of weights because same random seed used

import math
import numpy as np
import sys
from PIL import Image
import matplotlib.pyplot as plt 

#for shuffling data
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

In [2]:
#########
#MLP

# initialisation of the weights he_normal
def weights(noHiddenLayers,sizeOfLayers):

    W=[]
    b=[]

    for i in range(0,noHiddenLayers+1):
        
        W.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+sizeOfLayers[i])),
                                   (sizeOfLayers[i+1],sizeOfLayers[i])) )
        b.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+1)),
                                   (sizeOfLayers[i+1],1)) )

    W=np.array(W)
    b=np.array(b)
    
    return W,b

#mlp forward pass
#layer
def layer(w,x,b):
    out = np.matmul(w,x)+b
    return out

def apply_activationMLP(Activation_function,inp):
    
    #activation functions
    if Activation_function == 'relu':
        return np.where(inp<0,0,inp)
    elif Activation_function == "tanh":
        return np.tanh(inp)
    elif Activation_function == "sigmoid":
        return 1.0/(1+np.exp(-1.0*inp))
    elif Activation_function == "softmax":
        return (1.0/(np.sum(np.exp(inp),axis=0)))*(np.exp(inp))

#forward path
def forward_path(noHiddenLayers,X,W,b,Actfnvect):

    out=[]
    
    z=apply_activationMLP(Actfnvect[0],np.array(layer(W[0],X,b[0])))
    out.append(np.array(z))

    for i in range(1,noHiddenLayers):
        z=apply_activationMLP(Actfnvect[i],np.array(layer(W[i],out[i-1],b[i])))
        out.append(np.array(z))

    if noHiddenLayers > 0:
        z=apply_activationMLP(Actfnvect[-1],np.array(layer(W[-1],out[-1],b[-1])))
        out.append(np.array(z))

    y_pred = out[-1]

    return np.array(out),np.array(y_pred)


In [3]:
#only to import data
from keras import backend as K
import keras
from keras.datasets import mnist
 
    
################################

# #import data
iris_data = load_iris() # load the iris dataset

x = iris_data.data
y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
# Split the data for training and testing
x_train , x_test, y_train , y_test = train_test_split(x, y, test_size=0.20)

print(x_train.shape)
print(y_train.shape)

Using TensorFlow backend.


(120, 4)
(120, 3)


In [4]:
# run MLP algorithm

x_train = np.moveaxis(x_train,0,-1)
y_train = np.moveaxis(y_train,0,-1)
x_test = np.moveaxis(x_test,0,-1)
y_test = np.moveaxis(y_test,0,-1)

print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

(4, 120)
(3, 120)
(4, 30)
(3, 30)


In [5]:
#derivative of relu
def der_relu(x):
    return np.where(x == 0,0,1)

# backpropation
def backprop(y,y_true,z,T,u,M,x):
    
    dy = (y-y_true)
    
    # layer 3
    dT  = np.matmul(dy,z.T)
    db3 = np.sum(dy,axis=1).reshape(-1,1)
    
    # layer 2
    s   = np.matmul(T.T,dy)
    s   = s*der_relu(z)
    dM  = np.matmul(s,u.T)
    db2 = np.sum(s,axis=1).reshape(-1,1)
    
    # layer 1
    sm  = np.matmul(M.T,s)
    sm  = sm*der_relu(u)
    dW  = np.matmul(sm,x.T)
    db1 = np.sum(sm,axis=1).reshape(-1,1)
    
    return dW,dM,dT,db1,db2,db3

In [6]:
###
# Training parameters
num_classes = 3
epochs = 600
learning_rate_t=1e-4
learning_rate_0=5e-4
batch_size = 10

################################
#MLP PARAMETERS
noHiddenLayers=2

#also includes the input vector dimension and output vector dimension
sizeOfLayers=[x_train.shape[0],10,10,num_classes]

sizeofOutput=num_classes

Actfnvect = ['relu','relu','softmax']

In [7]:
W,b = weights(noHiddenLayers,sizeOfLayers)


In [8]:
# (a)
# stochastic gradient descent

# updating the weights

t=0
thresh=150

for i in range(epochs):
    loss=0
    for j in np.arange(0,x_train.shape[1],batch_size):       
        dW = np.zeros(W[0].shape)
        dM = np.zeros(W[1].shape)
        dT = np.zeros(W[2].shape)
        db1 = np.zeros(b[0].shape)
        db2 = np.zeros(b[1].shape)
        db3 = np.zeros(b[2].shape)
        for k in range(0,batch_size):
            # forward pass
            x = x_train[:,j+k].reshape(-1,1)
            y = y_train[:,j+k].reshape(-1,1)
            out,y_pred=forward_path(noHiddenLayers,x,W,b,Actfnvect)
            # backpropagation
            dWtemp,dMtemp,dTtemp,db1temp,db2temp,db3temp=backprop(y_pred,y,out[1],W[-1],out[0],W[1],x)
            
            dW=dW+dWtemp
            db1=db1+db1temp
            
            dM=dM+dMtemp
            db2=db2+db2temp
            
            dT=dT+dTtemp
            db3=db3+db3temp
        
            # calculate the loss
            loss = loss + (-1.0*np.sum(y*np.log(y_pred)))
         
        if t<thresh:
            alpha=(t*1.0/thresh)
            learning_rate=(1-alpha)*learning_rate_0+alpha*learning_rate_t
        
        t=t+1
        # Updating the weights SGD
        b[0]=b[0]-learning_rate*db1*(1.0/batch_size)
        W[0]=W[0]-learning_rate*dW*(1.0/batch_size)

        b[1]=b[1]-learning_rate*db2*(1.0/batch_size)
        W[1]=W[1]-learning_rate*dM*(1.0/batch_size)

        b[2]=b[2]-learning_rate*db3*(1.0/batch_size)
        W[2]=W[2]-learning_rate*dT*(1.0/batch_size)
    
    #print the loss in each epoch
    print('Epoch:'+str(i)+'         Loss:'+str(loss))

Epoch:0         Loss:256.0180606197502
Epoch:1         Loss:244.5094803739828
Epoch:2         Loss:235.3456093907099
Epoch:3         Loss:227.79758307801575
Epoch:4         Loss:221.49130513743228
Epoch:5         Loss:216.25172767069634
Epoch:6         Loss:211.8703426177371
Epoch:7         Loss:208.20393411689798
Epoch:8         Loss:205.1620016660833
Epoch:9         Loss:202.65543963815816
Epoch:10         Loss:200.61408235065323
Epoch:11         Loss:198.99297008988898
Epoch:12         Loss:197.7507784905046
Epoch:13         Loss:196.6930988957051
Epoch:14         Loss:195.6637412462342
Epoch:15         Loss:194.65610249957598
Epoch:16         Loss:193.66800617494374
Epoch:17         Loss:192.70037749079341
Epoch:18         Loss:191.75257229565685
Epoch:19         Loss:190.8228518363041
Epoch:20         Loss:189.91108132588417
Epoch:21         Loss:189.01617298987983
Epoch:22         Loss:188.1376001877022
Epoch:23         Loss:187.27489383002967
Epoch:24         Loss:186.4297173375

Epoch:208         Loss:119.92548333023744
Epoch:209         Loss:119.64727968103611
Epoch:210         Loss:119.35970825281373
Epoch:211         Loss:119.05521933858945
Epoch:212         Loss:118.73871581052384
Epoch:213         Loss:118.41253854769631
Epoch:214         Loss:118.08218801825403
Epoch:215         Loss:117.75225510802912
Epoch:216         Loss:117.416451461138
Epoch:217         Loss:117.0743604207305
Epoch:218         Loss:116.7267297297642
Epoch:219         Loss:116.3748227980384
Epoch:220         Loss:116.02633804033151
Epoch:221         Loss:115.6761619281833
Epoch:222         Loss:115.323970298766
Epoch:223         Loss:114.97244729535929
Epoch:224         Loss:114.61906417852069
Epoch:225         Loss:114.263408331263
Epoch:226         Loss:113.9094706789291
Epoch:227         Loss:113.55901630654647
Epoch:228         Loss:113.21026110415757
Epoch:229         Loss:112.86260986688269
Epoch:230         Loss:112.51599366859034
Epoch:231         Loss:112.17062978684427
Epo

Epoch:418         Loss:80.51851278057349
Epoch:419         Loss:80.43938602139445
Epoch:420         Loss:80.36060780842838
Epoch:421         Loss:80.28223530825737
Epoch:422         Loss:80.20406444369841
Epoch:423         Loss:80.12626660411543
Epoch:424         Loss:80.04877463715766
Epoch:425         Loss:79.97181575629331
Epoch:426         Loss:79.8961132039735
Epoch:427         Loss:79.82120799268932
Epoch:428         Loss:79.74665652972932
Epoch:429         Loss:79.6724416310807
Epoch:430         Loss:79.59886563369666
Epoch:431         Loss:79.52584392128004
Epoch:432         Loss:79.45308357387017
Epoch:433         Loss:79.3811698962838
Epoch:434         Loss:79.30962566541773
Epoch:435         Loss:79.23843536058452
Epoch:436         Loss:79.16758437276937
Epoch:437         Loss:79.097058948487
Epoch:438         Loss:79.02684613708008
Epoch:439         Loss:78.95693960545422
Epoch:440         Loss:78.8872644046857
Epoch:441         Loss:78.81786261436888
Epoch:442         Loss

In [9]:
out,y_pred=forward_path(noHiddenLayers,x_test,W,b,Actfnvect)

In [10]:
def predict(y):
    return np.argmax(y)

yvect=[]
y_trurevect=[]

for i in range(0,x_test.shape[1]):
    yvect.append(predict(y_pred[:,i]))
    y_trurevect.append(predict(y_test[:,i]))

# find accuracy
from sklearn.metrics import accuracy_score
#predicting test accuracy
print(accuracy_score(y_trurevect, yvect))

1.0


In [11]:
# to see the output vs true values

print y_trurevect
print yvect

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]


In [ ]:
print learning_rate